<a href="https://colab.research.google.com/github/dilanHewawitharana/Intro-to-Deep-Learning-with-PyTorch/blob/master/cassava_leaf_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets --upgrade --quiet

In [2]:
import opendatasets as od

In [3]:
od.download('https://www.kaggle.com/c/cassava-leaf-disease-classification')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dilansarith
Your Kaggle Key: ··········


  0%|          | 8.00M/5.76G [00:00<01:14, 83.3MB/s]

100%|██████████| 5.76G/5.76G [01:24<00:00, 73.0MB/s]

In [8]:
import zipfile

with zipfile.ZipFile('/content/cassava-leaf-disease-classification/cassava-leaf-disease-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('./cassava-leaf-disease-classification')

In [16]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import f1_score
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid
%matplotlib inline

In [17]:
DATA_DIR = './cassava-leaf-disease-classification'

TRAIN_DIR = DATA_DIR + '/train_images' # Contains training images
TEST_DIR = DATA_DIR + '/test_images' # Contains test images

TRAIN_CSV = DATA_DIR + '/train.csv' # Contains real labels for training images
TEST_CSV = '/content/cassava-leaf-disease-classification/sample_submission.csv' # Contains dummy labels for test image

In [18]:
!head "{TRAIN_CSV}"

image_id,label
1000015157.jpg,0
1000201771.jpg,3
100042118.jpg,1
1000723321.jpg,1
1000812911.jpg,3
1000837476.jpg,3
1000910826.jpg,2
1001320321.jpg,0
1001723730.jpg,4


In [19]:
!head "{TEST_CSV}"

image_id,label
2216849948.jpg,4


In [20]:
!ls "{TRAIN_DIR}" | head

1000015157.jpg
1000201771.jpg
100042118.jpg
1000723321.jpg
1000812911.jpg
1000837476.jpg
1000910826.jpg
1001320321.jpg
1001723730.jpg
1001742395.jpg


In [21]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [28]:
len(train_df)

21397

In [30]:
print(len(os.listdir(TRAIN_DIR)))

21397


In [34]:
label_map = pd.read_json('/content/cassava-leaf-disease-classification/label_num_to_disease_map.json', 
                         orient='index')
label_map

,0
0,Cassava Bacterial Blight (CBB)
1,Cassava Brown Streak Disease (CBSD)
2,Cassava Green Mottle (CGM)
3,Cassava Mosaic Disease (CMD)
4,Healthy
